## Reclass NaN values in rasters

Some sources provide rasters with NaN instead of a numeric NoData value. Although this seems a perfectly logical thing to do, these rasters are difficult to work with in most programs. The basic reason for this is that mathematically NaN != NaN so the areas which should be shown as not having data tend not to be, but are instead shown as the true numeric value of NaN which is somewhat implementation specific. 

So we want to re-class all NaNs in the data to a numeric NoData value which can then be successfully used as a mask. For the same reason, this is slightly tricky but at least working directly with the arrays we can take a bit more control. (whereas the Reclass tool in Arc doesn't really work, for example). 

One example of where we've had to do this was with the Facebook HRSL population grids as downloaded from HDX.

In [1]:
from raster_utilities.io.TiffFile import SingleBandTiffFile, RasterProps
from raster_utilities.utils.raster_tiling import getTiles

In [2]:
import os, glob

In [3]:
import numpy as np

In [4]:
files = glob.glob(r'C:\Temp\dataprep\population_af_2018-10-01\*.tif')
newNDV = -9999

In [7]:
files[26:]

['C:\\Temp\\dataprep\\population_af_2018-10-01\\population_AF27_2018-10-01.tif',
 'C:\\Temp\\dataprep\\population_af_2018-10-01\\population_AF28_2018-10-01.tif']

In [15]:
for f in files[25:]:
    fOut = (os.path.join(os.path.dirname(os.path.dirname(f)), "reclassed", os.path.basename(f)))
    if os.path.exists(fOut):
        continue
    t = SingleBandTiffFile(f)
    shape = t.GetShape()
    tiles = getTiles(shape[1], shape[0], 40000)
    tOut = SingleBandTiffFile(fOut)
    pIn =t.GetProperties()
    pOut = RasterProps(pIn[0], pIn[1], newNDV, pIn[3], pIn[4], pIn[5], pIn[6])
    tOut.SetProperties(pOut)
    for tile in tiles:
        arrIn = t.ReadForPixelLims(xLims=tile[0], yLims=tile[1])[0]
        #arrIn = t.ReadAll()[0]
        arrIn[np.isnan(arrIn)]=newNDV
        tOut.SavePart(data=arrIn, 
                     outOffsetYX=(tile[1][0], tile[0][0]),
                     cOpts=["COMPRESS=DEFLATE", "BIGTIFF=YES", "TILED=YES", "NUM_THREADS=ALL_CPUS"])
        del(arrIn)
    print(f)

C:\Temp\dataprep\population_af_2018-10-01\population_AF26_2018-10-01.tif
C:\Temp\dataprep\population_af_2018-10-01\population_AF27_2018-10-01.tif
C:\Temp\dataprep\population_af_2018-10-01\population_AF28_2018-10-01.tif
